In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("COMP4107").master("local[*]").getOrCreate()

In [3]:
df = spark.read.csv("data/btcusd_1-min_data.csv", header=True, inferSchema=True)
df_filtered = df.filter(df["high"] > 5)

In [4]:
df_filtered.show()

+------------+----+----+----+-----+----------+
|   Timestamp|Open|High| Low|Close|    Volume|
+------------+----+----+----+-----+----------+
| 1.3256091E9|5.32|5.32|5.32| 5.32|2.41917293|
|1.32560916E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560922E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560928E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560934E9|5.32|5.32|5.32| 5.32|       0.0|
| 1.3256094E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560946E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560952E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560958E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560964E9|5.32|5.32|5.32| 5.32|       0.0|
| 1.3256097E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560976E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560982E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560988E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32560994E9|5.32|5.32|5.32| 5.32|       0.0|
|   1.32561E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32561006E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32561012E9|5.32|5.32|5.32| 5.32|       0.0|
|1.32561018E9

In [5]:
spark.stop()